In [ ]:
# RUN THIS CELL FOR FORMAT
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

# Milestone 1
@merlionctc

**Table of Contents**
- [1. Introduction](#1.-Introduction)
  - [1.1 Derivative](#1.1-Derivatives)
  - [1.2 Automatic Differentiation](#1.2-Auto-Differentiation)
- [2. Background](#2.-Background)
  - [2.1 Chain Rule](#2.1-Chain-Rule) 
  - [2.2 Elementary functions](#2.2-Elementary-functions) 
  - [2.3 Forward mode](#2.3-Forward-mode) 
  - [2.4 Reversed mode](#2.4-Reversed-mode) 
  - [2.5 Dual Number](#2.5-Dual-Number) 
- [3. Usage](#3.-Usage)
  - [3.1 Installation](#3.1-Installation)
  - [3.2 How to use](#3.2-How-to-Use)
- [4. Software organization](#4.-Software-Organization)
- [5. Implementation](#5.-Implementation)
- [6. Reference](#6.-Reference)
- [7. Feedback](#7.-Feedback)




## 1. Introduction

We developed this package, `AutoDiff`,  in the light of automatic differentiation. The package can help to automatically differentiate a function input into the program. The package includes modules of forward-mode differentiation and backward-mode differentiation.

### 1.1 Derivatives

Formally, for single variable case, the derivative of a function, if it exists, is defined as

$$ f'(x) = \lim_{h\to0} \frac{f(a+h) - f(a)}{h} $$

to visualize, is the slope of the tangent line to the graph of the function at that point. The tangent line is the best linear approximation of the function near that input value.

Of course, derivatives may be generalized to functions of several real variables, and derivatives are useful in finding the maxima and minima of functions. Derivatives has a variety of applications in statistics and machine learning, and the process of finding a derivative is called *differentiation*.

There are three ways of differentiation realized in computer science:
- Numerical differentiation
- Symbolic differentiation
- Automatic differentiaton


### 1.2 Auto Differentiation
Automatic differentiation (AD), also called as algorithmic differentiation, is a set of techniques for efficiently and accurately evaluating derivatives of numeric functions expressed as computer programs <sup>1</sup>. It is not numerical differentiation, since numerical differentiation is the finite difference approximation of derivatives using the values of the original function evaluated at some sample points<sup>2</sup>. It is different from symbolic differentiation, since symbolic differentiation is the automatic manipulation of expressions for obtaining derivative expression<sup>3</sup>.

The essence of AD is that all numerical computations are ultimately compositions of a finite set of elementary operations for which the derivatives are easily known<sup>4</sup>. The algorithm of AD breaks down a function by looking at the sequence of elementary arithmetic operations (addition, subtraction, multiplication and division) and elementary functions (exponential, logrithmatic, and trigonometry). By applying the chain rule repeatedly to these operations, derivatives of arbitrary order can be computed automatically, accurately to machine accuracy.

This differentiation technique well-established and used with applications in different areas such as fluid dynamics, astronomy, and engineering design optimization.

To sum up, there are two major advantages of using AD:
- Computes derivatives to machine precision.
- Does not rely on extensive mathematical derivations or expression trees, so it is easily applicable to a wide class of functions.

## 2. Background

Automatic differentiation relies on several vital mathematical foundations, some of which will be illustrated at this part. Based on these conceptions, it will be more resonable for users to understand the software.

### 2.1 Chain Rule

Chain Rule is the most important concept in AD. It enables us to deal with complex functions with several layers and arguments. With implementing chain rule, we can easily divide the original complicated functions into basic parts made up with elementary functions, of which we will know the concrete derivative expressions.

Suppose there is a function $h\left(u\left(t\right)\right)$ and in order to calculate derivative of $h$ with respect to $t$, we should use chain rule. The derivative is $$\dfrac{\partial h}{\partial t} = \dfrac{\partial h}{\partial u}\dfrac{\partial u}{\partial t}.$$

In general, if a function $h$ has several arguments, or even its argument is a vector, so that $h = h(x(t))$ where  $x \in R^n$ and $t \in R^m $. In this way, $h$ is now the combination of $n$ functions, each of which has $m$ variables. The derivative of $h$ is now

 $$\nabla_{t}h = \sum_{i=1}^{n}{\frac{\partial h}{\partial x_{i}}\nabla y_{i}\left(t\right)}.$$

### 2.2 Elementary functions

Any complex function is made up with several elementary functions. As discussed above, we use chain rule to break it down and then focus on elementary functions to calulate their derivatives. 

In mathematics, an elementary function is a function of a single variable composed of particular simple functions. Elementary functions are typically defined as a sum, product and/or composition of many polynomials, rational functions, trigonometric and exponential functions, and their inverse functions.<sup>5</sup>

On the other hand, we know the concrete mathematical expression of the elementary functions, which will be used directly in the later graph structure of calculations. 


### 2.3 Forward mode

#### 2.3.1 Evaluation trace

Take the example of $g = (x+y)*z$, we will first demonstrate the evaluation trace and then its corresponding computational graph.

Let's evaluate g at the point (1,1,1). In the evaluation trace table, we will record the trace of each step, its  elementary operation as well as the corresponding numeric value at the point.

| Trace | Elementary Operation | Numeric Value |
| ----- | -------------------- | ------------- |
| $x$   | 1                    | 1             |
| $y$   | 1                    | 1             |
| $z$   | 1                    | 1             |
| $p$   | $x+y$                | 2             |
| $f$   | $v_1*z$              | 2             |

#### 2.3.2 Computational graph

The above evaluation trace can be easily visualized with the computational graph below. The node will represent the trace and the edge will represent the elementary operation.

<img src="forward_mode.png">

*Note: If you cannot see the image, please right click and open image in new tab*

#### 2.3.3 Explanation
The evaluation trace above is just the path we will follow in forward mode. On top of that, we will also carry the derivatives. And we will take the derivative of g on x.

| Trace | Elementary Operation | Numeric Value | Deri. on x    | Deri. Value on x |
| ----- | -------------------- | ------------- | ------------- | ---------------- |
| $x$   | 1                    | 1             | 1             | 1                |
| $y$   | 1                    | 1             | 0             | 0                |
| $z$   | 1                    | 1             | 0             | 0                |
| $v_1$ | $x+y$                | 2             | $\dot{x}$     | 1                |
| $f$   | $v_1*z$              | 2             | $\dot{v_1}*z$ | 1                |

### 2.4 Reverse mode

#### 2.4.1 Explanation

It should be noticed that in forward mode, chain rule is not utilized. We just follow the evaluation trace and combine the derivatives of elementary functions together. But for reverse mode, we will implement chain rule. 

However, it is important to realize that the reverse mode also requires the evaluation trace on forward mode to have the derivatives on the elementary functions. Then we will use chain rule to reversely calculate the final derivative.

The steps we use to implement reverse mode based on the evaluation trace in forward mode is as follows.

- STEP1: Start with $v_1$

   $$\overline{v_1} = \dfrac{\partial f}{\partial v_1} = 1.$$

- STEP2: Use chain rule to calculate$\overline{x}$

   $$\overline{x} = \dfrac{\partial f}{\partial v_1}\dfrac{\partial v_1}{\partial x}  = 1.$$

- STEP3: Get the derivative on x

   $$\overline{x} = \dfrac{\partial f}{\partial x}  = 1.$$

#### 2.4.2 Computational graph

In reverse mode, we just reversely implement chain rule to get the derivatives. And the computational graph for reverse mode is as follows.

<img src="reverse_mode.png">

*Note: If you cannot see the image, please right click and open image in new tab*

 ### 2.5 Dual Number
 
 A dual number has a real part and a dual part. Say we have $z = a + b\epsilon $. Then $a$ is the real part and $b$ is the dual part.

 For $\epsilon$, we define $\epsilon ^2 = 0$ but $\epsilon$ is not zero.
 
 Dual Number is really useful when we want to calculate derivatives of a function. For example, say we have

 $$ x = a + b\epsilon,   y = x^2$$
 
 Then we can derive
 
 $$y = (a + b\epsilon)^2 = a^2 + 2ab\epsilon + b^2\epsilon^2 = a^2 + 2ab\epsilon$$

 Therefore, it is really convenient to get the value of y from real part and get derivative of y from dual part. This is what we will implement in our codes.
 
 




## 3. Usage


 ### 3.1 Installation
 
The package will be distributed through PyPI (Not yet implemented, a usage example is shown on demo.py).

To install AutoDiff using pip:

 ```bash
 pip install AutoDiff
 ```

This will also install all required documents as dependency.

For now, to get started, please do git clone on our project and test by using `demo.py`

 ```bash
 git clone https://github.com/merlionctc/cs107-FinalProject.git
 ```
 
To ensure you have your enviroment and all required package setup, 

 ```bash
 pip install -r requirements.txt
 ```

To run a simple test case and demo we provided.

 ```bash
 python3 AutoDiff/src/autodiff/demo.py
 ```




### 3.2 How to Use

Here is an example that serves that a quick start tutorial.


```python
# The standard way to import AutoDiff:
from model import *
from dual_class import *
from elementary import *

# First Step: User instantiate variables
# val: value of variable that you start with
# der: value of the derivative of variable that you start with, usually starting with 1
# loc: The location/index this variable when there are multiple input variables for the target function(s). For example, if you initialize x1 first, the loc will be 0; then you initialize y1, the loc will increment to 1
# length: The length/number of the total variables that will be input when there are multiple input variables for the target function(s).For example, if you want to initialize x1,y1 and z1, the length will be 3, for each variable in the initialization process

x1 = Dual(val = 1, der=1, loc = 0, length = 3)
y1 = Dual(val = 2, der=1, loc = 1, length = 3)
z1 = Dual(val = 5, der=1, loc = 2, length = 3)

# Second Step: User inputs function, based on above variables
f1 = 3 * x1 + 4 * y1 * 2 - z1

# Third Step: User instantiate AutoDiff.Forward class 
fwd_test = Forward(f1)

# Four Step: User could choose to call instance method get_value() to get value of func
print(fwd_test.get_value())

# Five Step: User could choose to call instance method get_der() to get der of func
# Note: This method will return a derivative vector w.r.t to all variables 
print(fwd_test.get_der())

# Sixth Step: User could choose to call instance method get_der(var) to get der of func
# Note: This method will return a derivative vector w.r.t to specific variables you input
print(fwd_test.get_der(x1))
```


There are 2 classes inherited from `AutoDiff` and 3 public functions of this API:

`Forward(AutoDiff)`: Class that does forward mode differentiation of the function

`Reverse(AutoDiff)`: Class that does reverse mode differentiation of the function. (This is a future implementation)

`get_value()`: Getting value of differentation results

`get_der(*args)`: Getting derivative of all differentation results in a vector, or w.r.t to sepcific variable.

`get_jacobian()`: Getting Jacobian matrix of differentiation results. To get Jacobian, the input must be a 2-D numpy array, which gives an output of 2-D numpy array.

`get_expression()`: Getting a symbolic derivative expression of the function. (This is a future implementation)

Note: these are subject to change in the development process




## 4. Software Organization

Discuss how you plan on organizing your software package.

* Directory Structure

```
AutoDiff
│   README.md
│   .travis.yml  
│   LICENSE
│
└───src
    │
    └─── autodiff
    │   │   Func Parser (module, not yet implemented)
    │   │   interface.py (Interface, not yet implemented)
    │   │   model (AutoDiff main class with Forward/Reverse mode)
    │   │   elementary.py (Elementary Function class)
    │   │   dual_class.py (Dual Number Class)
    │   │   demo.py (A demo to usage of package)
│        ...
│
└───tests
│   │   autodiff_test.py
│   │   dual_class_test.py
│   │   elementary_test.py
│   │   Func Parser Test (*TBC)
│   │   Interface Test (*TBC)
│   │   ...
└───Docs
│   │   README.md
│   │   milestone1.ipynb
│   │   milestone2_progress.ipynb
│   │   milestone2.ipynb

```

* **Modules to Include**

 math: mathematical, algebric operations

 numpy: supports computations for large, multi-dimensional arrays and matrices. 

 [parser](https://docs.python.org/3.0/library/parser.html): (This is a future implementation). We will build a user interface in the future that asking user to input directly the expression she/he wants to work with. We will build on this standard library `parser` to parse 
 the function string into expression tree. Currently, this parser only handles parsing and evaluating basic arithmetic operations with numbers. 
 
 We will also use Numpy and Math for evaluating formulas.


* **Test Suite Design**

 We used Pytest as our test suite. We tested all of our implemented classes on every single instance methods.
 We also tested on special and corner cases such as simplication, real number etc. 
 The whole test suite is included in the *test* sub-directory. 
 And both TravisCI and Coveralls will be used to check the codes coverage and test integration.


* **Package Distribution**

  We will distribute our software using [Python Package Index (PyPi)](https://pypi.org/).
  Currently, we used Python project structure from [PyScaffold](https://pypi.org/project/PyScaffold/).

* **Software Package**

  We will package using the standard packaging tool ([setuptools](https://packaging.python.org/key_projects/#setuptools)), and following Package Python Projects [Tutorial](https://packaging.python.org/tutorials/packaging-projects/).

  






## 5. Implementation

Discuss how you plan on implementing the forward mode of automatic differentiation.

* **Core Data Structures**

  * The user will start by initializing variables using Dual class.

  * Numpy Array:
  Our core data structure of Dual class and its operation is built upon numpy array.
  The elementary function is also operated on numpy array and real numbers.
  We store each variable its corresponding value, derivative value in numpy array, such that we could apply elementary operation to entire array instead of just a scalar value.
  
  * For variables and differentiation point, it will be input in Dual number instantiation as its variable value, and value as the point of differentiation.
  User would also have to input the length of total variables and the position of current variable in dual class.

  * The function is directly built upon operation on dual class initialized above.
  
  * In the futre, We will also plan to use the above cited parser package to parse the formula. 
  Then a formula data structure will then be created, encoding the user's input as an abstract syntax tree. In this way, we will implement the evaluation and differentiation.


* **Implemented Classes**

  * Forward mode (AutoDiff): This class implements differention and calculate derivatives through forward mode.

  * Reverse mode (TBC*): This class will implement chain rule to calculate derivatives through reverse mode.

  * Elementary function: This class will overwrite and redefine elementary functions, making them applicable on Dual object.
 
  * `Dual`: dual number class. This class will take in any real number variable and construct and return it as a dual number,
  all the subsequent operations in AutoDiff will be done on Dual Number class
  The class also contains operations (addition, m) between dual numbers, and real numbers.
 
* **Method and Attributes of the Classes**

*Forward mode:*

```python
def __init__(self, f, var):
     self.f = f
     self.var = var
     
def get_value(self):
    """ Returns the value of f
        
        Parameters
        ----------
        self: Forward object
        
        Returns
        ------- 
        calculate the value of f on var through forward mode on specific value
        
        Examples
        -------- 
        >>> fwd.get_value()
    """
    return self.diff
     
def get_der(self, *args):
        """ Returns the derivative value of f
        
        Parameters
        ----------
        self: Forward object
        **kwargs: x, y etc. Dual Number

        Returns
        ------- 
        calculate the derivative of f on var through forward mode on all variables
        calculate the derivative of f on var through forward mode on specific variables
        
        Examples
        -------- 
        >>> fwd.get_der()
        >>> fwd.get_der(x1)
        """
        return result

def get_jacobian(self, **kwargs):
    # **kwargs: var = 'x','y' etc.
    #  calculate the derivative of f on var through forward mode with jacobian
    return self.diff
     
def get_expression(self, **kwargs):
    # **kwargs: var = 'x','y' etc.
    # return the list of derivative expression of the parsed formula.
    return self.express
```

   *Reverse mode: (A future implementation)*
    
    
```python
def __init__(self, f, var):
    self.f = f
    self.var = var
     
def get_value(self, **kwargs):
   # **kwargs: var = 'all','x','y' etc.
   # calculate the derivative of f on var through reverse mode to specific value
    return self.f.val

def get_jacobian(self, **kwargs):
   # **kwargs: var = 'all','x','y' etc.
   # calculate the derivative of f on var through reverse mode with jacobian
    return self.f.der
     
def get_jacobian(self, **kwargs):
   # **kwargs: var = 'all','x','y' etc.
   # calculate the derivative of f on var through reverse mode with jacobian
    return self.diff
     
def get_expression(self, **kwargs):
   # **kwargs: var = 'all','x','y' etc.
   # return the list of derivative expression of the parsed formula.
    return self.express
```

* **Dual Class** :
  
  The class instance itself has two main attributes: the value and the evaluated derivatives with respect to each input.
  And within the class we redefine some of the elementary functions and basic algebraic operations, in order to calculate both evaluation values and derivation.
  
  1. Attributes

  self.val: The evaluation values of the functions or the initialized values for the variable(dual object)
  self.der: The derivatives of the functions or the initialized derivative of the variable. When there are multiple variables to be input in a function. 
 
```python

def __init__(self, real_number):
    self.val = real_number
    self.der = 1
```


  * **Elementary Function** :
  
  For the elementary function, we write our own method of computing the value so that these function can be applied on the Dual number, as well as on the real number. 
 For example, in our daily usage, sine funtion on a real number `x` can be calculated via `NumPy` (np.sin(x)), but here when we calculate sine value for a Dual object, 
 we cleverly store the value (same as we got from np.sin(x)) and the derivative part. 

 We have implemented the following elementary functions and we provide a demo function for reference.
```python
#the function we have implemented so far

exp() #extend the exponential function to Dual number
log() #extend the logrithmatic function with base e to Dual number
sqrt() #extend the square root function with base e to Dual number
sin() #extend the sine function with base e to Dual number
cos() #extend the cosine function with base e to Dual number
tan() #extend the tangent function with base e to Dual number
arcsin() #extend the inverse sine function with base e to Dual number
arccos() #extend the inverse cosine function with base e to Dual number
arctan() #extend the inverse tangent function with base e to Dual number


 ##demo function
def sin(dual):
    """Calculate the sine of the input
        Keyword arguments:
        dual -- a dual number or a real number
        Return:
        the sine value
    """
    if isinstance(dual, Dual):
        der = np.cos(dual.val)*dual.der
        val = np.sin(dual.val)
        return Dual(val,der)
    else:
        return np.sin(dual)
             
```
If we call the above function, it will give the following output. 
```python
#...import necessary dependencies
>>> x = Dual(np.pi, 1)
>>> z = sin(x)
>>> print(z)
Dual(value=1.2246467991473532e-16, derivative=-1.0)

```
Notice z is a Dual object. This function also applies to real number:
```python
#...import necessary dependencies
>>> x_real = np.pi
>>> z_real = sin(x_real)
>>> print(z_real)
1.2246467991473532e-16

```

In the future, we may consider extending our elementary function libraries, including the hypobolic sine and cosine, as well as functions like csc and cot, log with base 2 and 10, if needed.

* **External Dependencies**

We have the following external libraries/Modules to include:

`NumPy`: This provides an API for a large collection of high-level mathematical operations. In addition, it provides support for array operations.

`Math`: This provides access to some mathematical functions also.

`parser`: (This is a future implementation)we will use standard library `parser` and extend it to parse the function when user enter as string, in the interface we will have built.

`pytest`: This is the way we perform testing on our codes.

Also, we include Travis CI and CodeCov to make sure that the validility of building and the code. 
  
* **Future implementation**

Please refer to section 6. Future Feature.




## 6. Future Features

* **Implement User Interface with elegant input**

  Now that our package requires the user to initialize the variables using Dual class with specified location and length of the input variables. 
  And then user could use these Dual objects to create the target functions. However, there are mainly two future improvements that can be implemented to make the package more friendly to users.

  1. With user entering the function expression, our package will automatically parse the function break it down into elementary functions and store them into tree structure.

  When users want to repeatively use the functions that require our package to calculate the deriavtives, now the users have to redefine the funtions repeatively.
  Once we could parse the function expressions and use tree structure to store the operations and input variables at each step, we can free users from initializing functions and variables repeatively.

  2. With user directly entering the input variables for the target functions, our package will automatically initialize the Dual objects that will be used in the target functions.
  
  When users initialize the variables, they should first think about how many variables will be used in the target functions. But sometimes users may want to use the variables later in different functions.
  Therefore, in the future we will use list to store the values that the user is intended to use in the function instead of the initialized variables. Our package will automatically initialize the variables for the user
  with these values.

* **Jacobian of multiple functions and multiple variables**

  Now our package could deal with single function with multiple variables. But in the future, we will implement the case when the use will use multiple functions with multiple variables.
  In this way, we will allow the value and derivatives to be array and combine the derivatives of different functions and variables together.

*  **Reverse Mode of Auto Differentiation class**

  Now our package could calculate deriavtives through forward mode of Auto Differentiation. However, in many tasks in machine learning, it is important to implement reverse mode to customize gradient descent in these problems.
  To create class of reverse mode, we will modify the dual class into cantaining the input of each new-generated dual objects through operation. In this way, we can create a linked graph of this objects as nodes and then we can 
  sort them in topological order to get the reverse path. Then we can calculate the derivatives of the previous traces/objects of the current object/trace to gain the derivative of current object.

*  **Expression and Visualization**

  If time allows, we also plan to actualize expression and symbolic representation of differentiation. This would requires parsing of input function, 
  and store them in a tree (or other neccessary data structure), and recombine symbolic representation together after finishing differnetiation.
  It could requires higher complexity.
  Besides, if possible, we would also plan to visualize our computational graph in python via visualization package, e.g. d3. 





## 7. Reference

[[1]](https://www.jmlr.org/papers/volume18/17-468/17-468.pdf): Baydin, Atilim Gunes; Pearlmutter, Barak; Radul, Alexey Andreyevich; Siskind, Jeffrey (2018). "Automatic differentiation in machine learning: a survey". Journal of Machine Learning Research. 18: 1–43.

[[2]](https://fac.ksu.edu.sa/sites/default/files/numerical_analysis_9th.pdf):Rirchard L. Burden and J. Douglas Faires. Numerical Analysis. Brooks/Cole, 2001.

[[3]](https://www.springer.com/gp/book/9783540654667):Johannes Grabmeier and Erich Kaltofen. Computer Algebra Handbook: Foundations, Applications, Systems. Springer, 2003

[[4]](https://www.jstor.org/stable/24103956): Arun Verma. An introduction to automatic differentiation. Current Science, 78(7):804–7,
2000.

[[5]](https://www.worldcat.org/oclc/31441929):  Spivak, Michael. (1994). *Calculus* (3rd ed.). Houston, Tex.: Publish or Perish. p. 359.


## 8. Feedback

### 8.1 Milestone1

* Your document looks professional!Your introduction and background are great. And your following demo codes are neat and clear. 

  Thank you!
 

* It's nice to include the explanations of reverse mode if you want to implement it as an additional feature. 

  We re-organized our background part into the elementary function, the chain rule, the forward mode, the backward mode, and the dual number class. The forward mode section contains explanations and illustration of trace table and graph,
 the backward mode also contains explanation, and graphical illustrations of operation. The dual number class is a new feature, we included because we think this is a bettter and efficient way to 
 deal with elementary functions. We also included chain rule explanation as before.
 

* Implementation: it would be better if you could elaborate on how to deal with elementary functions, e.g. by giving some demos.

  We add an example code of sin function to illustrate how we are dealing with elementary functions, notice that in the newest version we relate this to the dual number class.